In [2]:
# gymのインポート
import gymnasium as gym
# pandasのインポート
import pandas as pd
# matplotlibのインポート
import matplotlib.pyplot  as plt
import pygame
import sys
from collections import deque
import random
import numpy as np
import gymnasium as gym
# ニューラルネットワークモデルのインポート
from sklearn.neural_network import MLPRegressor
# ACAgentクラスの作成
from sklearn.exceptions import NotFittedError
import joblib

In [35]:
import math
def dist(a, b):
    a1,a2 = a[0],a[1]
    b1,b2 = b[0],b[1]
    z_2 = (a2 - a1) ** 2 + (b2 - b1) ** 2
    z = math.sqrt(z_2)
    return z

In [14]:
from Fighter import Fighter


In [ ]:
class ACAgent(object):
    def __init__(self, actions): # 初期化メソッド
        self.actions = actions
        self.model = None
        self.initialized = False

    def initialize(self): # モデルを初期化するメソッド
        # モデルは中間層が1層で変数の数を32個
        actor = MLPRegressor(hidden_layer_sizes=(32,), max_iter=1) # Actorのモデル
        critic = MLPRegressor(hidden_layer_sizes=(32,), max_iter=1) # Criticのモデル
        self.model = {'actor':actor, 'critic':critic} # ActorとCriticを辞書型として持っておく
        self.initialized = True # 初期化フラグをTrueにしておく

    def policy(self, state): # 状態を渡して行動を選択するメソッド
        estimated = self.estimate(state)
        prob_list = [np.exp(q)/np.exp(estimated).sum() for q in estimated]
        prob_sum = sum(prob_list)
        prob_list_normalized = [prob / prob_sum for prob in prob_list]
        # print(sum(prob_list_normalized))
        action = np.random.choice(self.actions, size = 1, p = prob_list_normalized)[0]
        return action

    def load_models(self):
        models = {}
        models['actor'] = joblib.load('actor.pkl') # actorのモデルを読み込む
        models['critic'] = joblib.load('critic.pkl') # criticのモデルを読み込む
        self.model = models
        self.initialized = True # 初期化フラグをTrueにしておく

    def update(self, experience, gamma): # 経験を蓄積したデータと割引率を渡して学習を行うメソッド
        # 蓄積した経験において現在の状態と遷移先の状態の組を作る
        states = [] # 現在の状態
        new_states = [] # 遷移先の状態
        for ex in experience:
            states.append(ex['state'])
            new_states.append(ex['new_state'])
        states = np.concatenate(states, axis=0) # (n, 4)のnumpy.arrayとした
        new_states = np.concatenate(new_states, axis=0) # (n, 4)のnumpy.arrayとした

        # criticの学習
        try: # partial_fitする前にpredictはできないため例外処理を実装する
            estimated_values = self.model['critic'].predict(new_states) # 現在の状態に対する新しい価値評価の見積もり(n,)
            for i, ex in enumerate(experience):
                value = ex['reward']
                if not ex['done']: # doneフラグがFalseの時(棒が倒れていない時)次の状態がある
                    value += gamma*estimated_values[i]
                estimated_values[i] = value
        except NotFittedError:
            estimated_values = np.random.random(size=len(states))
        self.model['critic'].partial_fit(states, estimated_values) # 新しい価値の見積もりに近い出力になるように学習

        # actorの学習
        try: # partial_fitする前にpredictはできないため例外処理を実装する
            estimated_action_values = self.model['actor'].predict(states) # 現在の状態に対する価値評価(n,len(self.actions))
        except NotFittedError:
            estimated_action_values = np.random.random(size=(len(states), len(self.actions)))
        for i, ex in enumerate(experience): # とった行動に対して新しい価値評価の見積もりに変える
            estimated_action_values[i, ex['action']] = estimated_values[i]
        self.model['actor'].partial_fit(states, estimated_action_values) # 新しい価値の見積もりに近い出力になるように学習

    def estimate(self, state): # 状態を渡して各行動の価値評価を推定するメソッド
        if self.initialized:
            return self.model['actor'].predict(state)[0] # (1,len(self.actions))の形で返るので、(len(self.actions),)で出力する
        else:
            return np.random.random(size=len(self.actions)) # 初期化フラグがFalseの時はランダムな値を出力する


In [36]:
import gymnasium as gym
import numpy as np
import pygame
from pygame import gfxdraw
import math
import random
from typing import Optional
# reset(self) ：環境を初期状態にして初期状態(state)の観測(observation)をreturnする
# step(self, action) ：行動を受け取り行動後の環境状態(state)の観測(observation)・即時報酬(reward)・エピソードの終了判定(done)・情報(info)をreturnする
# render(self, mode) ：modeで指定されたように描画もしは配列をreturnする
# close(self) ：環境を終了する際に必要なクリーンアップ処理を実施する
# seed(self, seed=None) ：シードを設定する

# 使用するデータ型
# Discrete：[0, n-1]で指定したn個の離散値空間を扱う整数型（int）
# 使い方はDiscrete(n)
# Box：[low, high]で指定した連続値空間を扱う浮動小数点型（float）
# 使い方はBox(low, high, shape, dtype)
# lowおよびhighはshapeで与えたサイズと同じndarrayになります。
# 次に、TupleとDictの使い方について示します。

# Tuple：DiscreteやBoxなどの型をタプルで合体
# 使い方の例はTuple((Discrete(2), Box(0, 1, (4, ))))
# Dict：DiscreteやBoxなどの型を辞書で合体
# 使い方の例はDict({'position':Discrete(2), 'velocity':Box(0, 1, (4, ))})




class MyEnv(gym.Env):

    metadata = {
        "render_modes": ["human"],
        "render_fps": 20,
    }

    # def __init__(self,models,is_cpu,render_mode: Optional[str] = None):
    def __init__(self,render_mode: Optional[str] = None):
        # action_space ：エージェントが取りうる行動空間を定義
        # observation_space：エージェントが受け取りうる観測空間を定義
        # reward_range ：報酬の範囲[最小値と最大値]を定義

        self.screen = None
        # self.clock = None
        self.clock = pygame.time.Clock()
        self.window_x = 1000
        self.window_y = 700

        # self.models = models
        # self.is_cpu = is_cpu
        RIGIT_MAX = 20
        self.jump_speed = 30  # <= ジャンプの初速度
        self.gravity = 16
        self.move_speed = 10
        self.stage_pos = [0, 550]  # <= 表示するステージの位置
        self.size = [50,80]
        self.radius = 30

        self.player1 = None
        self.player2 = None

        # アクション数定義
        # 移動：「左」「右」「上」「移動なし」，攻撃：「する」「しない」
        ACTION_NUM=8
        self.action_space = gym.spaces.Discrete(ACTION_NUM)
        self.render_mode = render_mode

        # 状態の範囲を定義,inattackrangeが１のときはどちらもアタックできる距離にある
        # 水平距離，垂直距離，P1x,P1y,P2x,P2y,inatackrange,p1cooldown,p2cooldown,p1canjump,p2canjupm
        max_distancex = self.window_x - self.size[0]
        max_distancey = self.stage_pos[1] - self.size[1]
        max_x = self.window_x - self.size[0]
        max_y = self.stage_pos[1] - self.size[1]
        LOW = np.array([0,0,0,0,0,0,0,0,0,0,0])
        HIGH = np.array([max_distancex,max_distancey,max_x,max_y,max_x,max_y,1,RIGIT_MAX,RIGIT_MAX,1,1])
        self.observation_space = gym.spaces.Box(low=LOW, high=HIGH)
        # 即時報酬の値
        self.reward_range = (-5,5)
        self.reset()

    def reset(self):
        # 環境を初期状態にする関数
        # 初期状態をreturnする
        # リセットの際に、乱数seedのリセットはしてはいけないので注意してください。
        player1_pos = [600, 470]  # <= 操作キャラの位置
        direction1 =3 #キャラの方向，0=上,1=した,2=右.3=左

        player2_pos = [200, 470]
        direction2 =2 #キャラの方向，0=上,1=した,2=右.3=左

        self.player1 = Fighter(self.size, self.gravity, self.move_speed,self.jump_speed,player1_pos,direction1)
        self.player2 = Fighter(self.size, self.gravity, self.move_speed,self.jump_speed,player2_pos,direction2)

        #初期化
        observation=[player1_pos[0]-player2_pos[0],player1_pos[1]-player2_pos[1],player1_pos[0],player1_pos[1],player2_pos[0],player2_pos[1],0,0,0,1,1]
        return np.array(observation, dtype=np.float32), {}

    def step(self, action_index):
        # 行動を受け取り行動後の状態をreturnする
        # stepメソッドは、action_spaceで定義された型に沿った行動値を受け取り、環境を1ステップだけ進めます。
        # 進めた後の観測、報酬、終了判定、その他の情報を生成し、リターンします。
        # infoにはデバックに役立つ情報などを辞書型として格納することができます。
        # 唯一、自由に使える変数なので、存分にinfoを活用しましょう。

        # observation ：object型。observation_spaceで設定した通りのサイズ・型のデータを格納。
        # reward ：float型。reward_rangeで設定した範囲内の値を格納。
        # done ：bool型。エピソードの終了判定。
        # info ：dict型。デバッグに役立つ情報など自由に利用可能。

        done=False

        self.player1.controlfromAction(action_index)
        self.player2.controlrandom()
        # control_character_random(self.player2)

        self.player1.contact_judgment(self.player2)
        self.player2.contact_judgment(self.player1)

        self.player1.move()
        self.player2.move()

        self.player1.contact_judgment(self.player2)
        self.player2.contact_judgment(self.player1)

        self.player1.character_action(self.player2)
        self.player2.character_action(self.player1)

        reward = 0
        if any(self.player2.hit_judg):
            reward += 1
        if any(self.player1.hit_judg):
            reward -= 1

        if self.player1.damage >= 390:
            reward = -5
            done = True

        if self.player2.damage >= 390:
            reward = 5
            done = True

        self.player1.hit_action()
        self.player2.hit_action()

        self.player1.contact_judgment(self.player2)
        self.player2.contact_judgment(self.player1)

        inattackrange = 0
        if self.player1.pos_x > self.player2.pos_x:
            circle_pos = (self.player1.pos_x, self.player1.pos_y + self.player1.height // 2)
            enemy_hit_pos = (self.player2.pos_x + self.player2.pos_x / 2, self.player2.pos_y + self.player2.height / 2)
        else:
            circle_pos = (self.player1.pos_x + self.player1.width, self.player1.pos_y + self.player1.height // 2)
            enemy_hit_pos = (self.player2.pos_x, self.player2.pos_y + self.player2.height / 2)
        print(dist(circle_pos,enemy_hit_pos))
        if 0 <= dist(circle_pos,enemy_hit_pos) <= self.radius + self.player2.height / 2:
            inattackrange = 1

        p1canjump = 1 if self.player1.canMoveRange[0] == 0 else 0
        p2canjump = 1 if self.player2.canMoveRange[0] == 0 else 0
        # 水平距離，垂直距離，P1x,P1y,P2x,P2y,inattackrange,p1cooldown,p2cooldown,p1canjump,p2canjupm
        observation=[abs(self.player1.pos_x - self.player2.pos_x),
                     abs(self.player1.pos_y - self.player2.pos_y),
                     self.player1.pos_x,
                     self.player1.pos_y,
                     self.player2.pos_x,
                     self.player2.pos_y,
                     inattackrange,
                     self.player1.rigit_time,
                     self.player2.rigit_time,
                     p1canjump,
                     p2canjump]

        #ゲームが長いとマイナス
        if done == False:
            reward -= 0.1
        # 今回の例ではtruncatedは使用しない
        truncated = False
        # 今回の例ではinfoは使用しない
        info = {}
        return np.array(observation, dtype=np.float32),reward,done,truncated,info

    def render(self):
        if self.render_mode is None:
            return
        if self.screen is None:
            #初期化
            pygame.init()
            if self.render_mode == "human":
                pygame.display.init()
                self.screen = pygame.display.set_mode((self.window_x, self.window_y)) # ウィンドウサイズの指定
                # self.font = pygame.font.Font(None, 55)
            else: # mode == "rgb_array"
                self.screen = pygame.Surface((self.window_x, self.window_y))
        if self.clock is None:
            self.clock = pygame.time.Clock()

        # modeとしてhuman, rgb_array, ansiが選択可能
        # humanなら描画し, rgb_arrayならそれをreturnし, ansiなら文字列をreturnする

        self.surf = pygame.Surface((self.window_x, self.window_y))
        self.surf.fill((250, 250, 250))
        # text = self.font.render('Score:'+str(self.point), True, (255,255,255))   # 描画する文字列の設定
        # ステージの描画
        pygame.draw.rect(self.surf , (0, 250, 0), (self.stage_pos[0], self.stage_pos[1], 1500, 50))


        # プレイヤー1と2の描画(figureとaction.lifeの描画)
        player1_rect = (self.player1.pos_x, self.player1.pos_y, self.size[0], self.size[1])
        gfxdraw.rectangle(self.surf, player1_rect,(255,0,0))
        player2_rect = (self.player2.pos_x, self.player2.pos_y, self.size[0], self.size[1])
        gfxdraw.rectangle(self.surf, player2_rect,(255,0,0))

        # 攻撃の描画
        if self.player1.circle_pos is not None:
            gfxdraw.filled_circle(self.surf,self.player1.circle_pos[0],self.player1.circle_pos[0],30,(0,0,250))
        if self.player2.circle_pos is not None:
            gfxdraw.filled_circle(self.surf,self.player2.circle_pos[0],self.player2.circle_pos[0],30,(0,0,250))

        # lifeゲージの描画
        gfxdraw.rectangle(self.surf, (570, 20, 400, 30), (120, 120, 120))
        if self.player1.damage < 390:
            gfxdraw.rectangle(self.surf, (575, 25, 390 - self.player1.damage, 20),(250, 200, 0))
        gfxdraw.rectangle(self.surf, (30, 20, 400, 30), (120, 120, 120))
        if self.player1.damage < 390:
            gfxdraw.rectangle(self.surf, (35 + self.player2.damage, 25, 390 - self.player2.damage, 20),(250, 200, 0))

        self.surf = pygame.transform.flip(self.surf, False, False)
        self.screen.blit(self.surf, (0, 0))
        # self.screen.blit(text, [10, 10])# 文字列の表示位置
        if self.render_mode == "human":
            pygame.event.pump()
            self.clock.tick(self.metadata["render_fps"])
            pygame.display.flip()


        # if self.render_mode == "human":
        #     self.screen = pygame.display.set_mode((self.window_x, self.window_y)) # ウィンドウサイズの指定
        #     # pygame.time.wait(30)#更新時間間隔
        #     # pygame.display.set_caption("Pygame Test") # ウィンドウの上の方に出てくるアレの指定
        #     self.screen.fill((0,0,0,)) # 背景色の指定。RGBだと思う
        #     text = self.font.render('Score:'+str(self.point), True, (255,255,255))   # 描画する文字列の設定
        #     self.screen.blit(text, [10, 10])# 文字列の表示位置

        #     pygame.draw.rect(self.screen, (255,0,0), (self.rect_x,self.rect_y,self.rect_width,self.rect_height))#的の描画
        #     pygame.draw.circle(self.screen, (0,95,0), (self.ball_x,self.ball_y), 10, width=0)#ボールの描画
        #     # pygame.draw.aaline(self.screen, (255,0,255), (self.ball_x,self.ball_y), (self.ball_x_next,self.ball_y_next), 0)#バーの描画
        #     pygame.display.update() # 画面更新
        #     pygame.event.pump()
        #     self.clock.tick(self.metadata["render_fps"])
        #     pygame.display.flip()
        elif self.render_mode == "rgb_array":
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(self.screen)), axes=(1, 0, 2)
            )

    def close(self):
        pygame.quit()
    def seed(self, seed=None):
        pass



In [47]:
# Observerクラスの作成
class Observer(object):
    def __init__(self, env): # 初期化メソッド
        self.env = env
        self.observation_space = env.observation_space
        self.action_space = env.action_space

    def render(self): # 状態などを可視化するメソッド
        self.env.render()

    def reset(self): # 環境を初期化して初期状態を返すメソッド
        return self.preprocess(self.env.reset()[0])

    def step(self, action): # 行動を渡して前処理した状態と報酬などを返すメソッド
        print(action)
        state, reward, done, _, info = self.env.step(action)
        print(state)
        return self.preprocess(state), reward, done, info
    def preprocess(self, state):
        return state.reshape((1, 11))

In [48]:

# models = [model, None]
# is_cpu = [True, False]

In [49]:
env = MyEnv(render_mode="human")
observer = Observer(env) # Observer作成
state = observer.reset() # observerを初期化し、前処理済みの初期状態を返す

In [50]:
actions = [1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2]
for action in actions: # 定義した行動のリストを逐次的に入力していく
    new_state, reward, done, info = observer.step(action) # 行動を入力して進める
    print('\n行動:', action)
    print('報酬:', reward)
    print('状態:', new_state)
    print(observer.render()) # 状態の可視化
    if done: # 終了判定(done)がTrueとなった場合終了
        env.close()
        break


1
224.72205054244233
[390.   0. 590. 470. 200. 470.   0.  19.   0.   1.   1.]

行動: 1
報酬: -0.1
状態: [[390.   0. 590. 470. 200. 470.   0.  19.   0.   1.   1.]]
None
1
235.63743335896356
[390.   0. 580. 470. 190. 470.   0.  17.   0.   1.   1.]

行動: 1
報酬: -0.1
状態: [[390.   0. 580. 470. 190. 470.   0.  17.   0.   1.   1.]]
None
1
204.02205763103166
[380.  30. 570. 470. 190. 440.   0.  15.   0.   1.   0.]

行動: 1
報酬: -0.1
状態: [[380.  30. 570. 470. 190. 440.   0.  15.   0.   1.   0.]]
None
1
202.27703774773843
[360.  14. 560. 470. 200. 456.   0.  13.   0.   1.   0.]

行動: 1
報酬: -0.1
状態: [[360.  14. 560. 470. 200. 456.   0.  13.   0.   1.   0.]]
None
1
228.52789764052878
[360.   0. 550. 470. 190. 470.   0.  11.  19.   1.   1.]

行動: 1
報酬: -0.1
状態: [[360.   0. 550. 470. 190. 470.   0.  11.  19.   1.   1.]]
None
1
241.8677324489565
[360.   0. 540. 470. 180. 470.   0.   9.  17.   1.   1.]

行動: 1
報酬: -0.1
状態: [[360.   0. 540. 470. 180. 470.   0.   9.  17.   1.   1.]]
None
1
240.8318915758459
[350.   0

: 

In [ ]:
# Observerクラスの作成
class Observer(object):
    def __init__(self, env): # 初期化メソッド
        self.env = env
        self.observation_space = env.observation_space
        self.action_space = env.action_space

    def preprocess(self, state): # 状態に対する前処理メソッド
        pass

    def render(self): # 状態などを可視化するメソッド
        self.env.render()

    def reset(self): # 環境を初期化して初期状態を返すメソッド
        return self.preprocess(self.env.reset()[0])

    def step(self, action): # 行動を渡して前処理した状態と報酬などを返すメソッド
        state, reward, done, _, info = self.env.step(action)
        return self.preprocess(state), reward, done, info


# Agentクラスの作成
# AgentはObserverから渡される状態と報酬によって適切な行動を行うモジュール
class Agent(object):
    def __init__(self, actions): # 初期化メソッド
        self.actions = actions
        self.model = None
        self.initialized = False

    def initialize(self): # モデルを初期化するメソッド
        pass

    def load_models(self): # 訓練済みモデルをロードする
        pass

    def estimate(self, state): # 状態を渡して各行動の価値評価を推定するメソッド
        pass

    def update(self, experience, gamma): # 経験を蓄積したデータと割引率を渡して学習を行うメソッド
        pass

    def policy(self, state): # 状態を渡して行動を選択するメソッド
        estimated = self.estimate(state)
        prob_list = [np.exp(q)/np.exp(estimated).sum() for q in estimated]
        prob_sum = sum(prob_list)
        prob_list_normalized = [prob / prob_sum for prob in prob_list]
        # print(sum(prob_list_normalized))
        action = np.random.choice(self.actions, size = 1, p = prob_list_normalized)[0]
        return action

# Trainerは、ObserverとAgentの間でやり取りを行い、蓄積した経験をAgentに学習させ、学習の進捗(得られた報酬の履歴)を管理するモジュール
# Trainerクラスを作成
class Trainer(object):
    def __init__(self, num_episodes, gamma, buffer_length): # 初期化メソッド
        self.num_episodes = num_episodes
        self.gamma = gamma
        self.buffer_length = buffer_length
        self.experience = deque(maxlen=buffer_length)
        self.rewards = []

    def load(self, agent): # agentのモデルを読み込むメソッド
        pass

    def save(self, agent): # agentのモデルを保存するメソッド
        pass

    def begin_train(self, agent): # 学習の初めにモデルの初期化などを行うメソッド
        pass

    def batch_train(self, agent, batch_size): # 1ステップにおいてバッチサイズを渡してagentを学習するメソッド
        pass

    def train(self, agent, observer, batch_size): # agentとobserverとバッチサイズを渡して全体の学習を行うメソッド
        training = False # 学習を行うフラグ
        for episode in range(self.num_episodes):
            s = observer.reset() # observerを初期化し、前処理済みの初期状態を返す
            done = False # エピソードの終了フラグ
            reward_per_episode = 0 # 1エピソード当たりの報酬の総和
            while not done: # エピソードが終了しない間はずっと処理を行う
                action = agent.policy(s) # agentが戦略に従って行動を選択する
                new_state, reward, done, info = observer.step(action) # agentがとった行動に対してobserverが前処理済みの状態などを返す
                ex = {'state':s, 'action':action, 'reward':reward, 'new_state':new_state, 'done':done} # 経験: {'state': 現在の状態, 'action':行動, 'reward':報酬, 'new_state':遷移先の状態, 'done':終了フラグ}
                self.experience.append(ex) # appendメソッドで経験を蓄積
                reward_per_episode += reward # 獲得報酬を計算
                if not training: # 学習を行うフラグがFalseのとき
                    self.begin_train(agent)
                    training = True
                else: # 学習を行うフラグがTrueのとき
                    if len(self.experience) == self.buffer_length: # buffer_length分だけ経験が蓄積しているとき学習を行う
                        self.batch_train(agent, batch_size) # agentを学習
                s = new_state # 状態を更新
            self.rewards.append(reward_per_episode) # appendメソッドで獲得した報酬を格納


# CartPoleObserverクラスの作成
class MyFight(Observer):
    def preprocess(self, state):
        return state.reshape((1, 40))

class ACAgent(Agent):
    def initialize(self): # モデルを初期化するメソッド
        # モデルは中間層が1層で変数の数を32個
        actor = MLPRegressor(hidden_layer_sizes=(32,), max_iter=1) # Actorのモデル
        critic = MLPRegressor(hidden_layer_sizes=(32,), max_iter=1) # Criticのモデル
        self.model = {'actor':actor, 'critic':critic} # ActorとCriticを辞書型として持っておく
        self.initialized = True # 初期化フラグをTrueにしておく

    def load_models(self):
        models = {}
        models['actor'] = joblib.load('actor.pkl') # actorのモデルを読み込む
        models['critic'] = joblib.load('critic.pkl') # criticのモデルを読み込む
        self.model = models
        self.initialized = True # 初期化フラグをTrueにしておく

    def update(self, experience, gamma): # 経験を蓄積したデータと割引率を渡して学習を行うメソッド
        # 蓄積した経験において現在の状態と遷移先の状態の組を作る
        states = [] # 現在の状態
        new_states = [] # 遷移先の状態
        for ex in experience:
            states.append(ex['state'])
            new_states.append(ex['new_state'])
        states = np.concatenate(states, axis=0) # (n, 4)のnumpy.arrayとした
        new_states = np.concatenate(new_states, axis=0) # (n, 4)のnumpy.arrayとした

        # criticの学習
        try: # partial_fitする前にpredictはできないため例外処理を実装する
            estimated_values = self.model['critic'].predict(new_states) # 現在の状態に対する新しい価値評価の見積もり(n,)
            for i, ex in enumerate(experience):
                value = ex['reward']
                if not ex['done']: # doneフラグがFalseの時(棒が倒れていない時)次の状態がある
                    value += gamma*estimated_values[i]
                estimated_values[i] = value
        except NotFittedError:
            estimated_values = np.random.random(size=len(states))
        self.model['critic'].partial_fit(states, estimated_values) # 新しい価値の見積もりに近い出力になるように学習

        # actorの学習
        try: # partial_fitする前にpredictはできないため例外処理を実装する
            estimated_action_values = self.model['actor'].predict(states) # 現在の状態に対する価値評価(n,len(self.actions))
        except NotFittedError:
            estimated_action_values = np.random.random(size=(len(states), len(self.actions)))
        for i, ex in enumerate(experience): # とった行動に対して新しい価値評価の見積もりに変える
            estimated_action_values[i, ex['action']] = estimated_values[i]
        self.model['actor'].partial_fit(states, estimated_action_values) # 新しい価値の見積もりに近い出力になるように学習

    def estimate(self, state): # 状態を渡して各行動の価値評価を推定するメソッド
        if self.initialized:
            return self.model['actor'].predict(state)[0] # (1,len(self.actions))の形で返るので、(len(self.actions),)で出力する
        else:
            return np.random.random(size=len(self.actions)) # 初期化フラグがFalseの時はランダムな値を出力する

# CartPoleTrainerクラスの作成
class CartPoleTrainer(Trainer):
    def load(self, agent): # agentのモデルを読み込むメソッド
        models = {}
        models['actor'] = joblib.load('actor.pkl') # actorのモデルを読み込む
        models['critic'] = joblib.load('critic.pkl') # criticのモデルを読み込む
        agent.model = models
        agent.initialized = True

    def save(self, agent):
        joblib.dump(agent.model['actor'], 'actor.pkl') # actorのモデルを'actor.pkl'に保存する
        joblib.dump(agent.model['critic'], 'critic.pkl') # criticのモデルを'critic.pkl'に保存する

    def begin_train(self, agent): # 学習の初めにモデルの初期化などを行うメソッド
        agent.initialize() # モデルを初期化
        batch = np.random.choice(self.experience, size=1, replace=False) # 適当にサンプリング
        agent.update(batch, self.gamma) # 適当なデータで学習

    def batch_train(self, agent, batch_size): # 1ステップにおいてバッチサイズを渡してagentを学習するメソッド
        batch = np.random.choice(self.experience, size=batch_size, replace=False) # batch_size数分サンプリング
        agent.update(batch, self.gamma) # サンプリングされたデータで学習



if __name__ == "__main__":
    # CartPole-v0の攻略
    # cart_pole_env = gym.make('CartPole-v0',render_mode='human')  # 環境作成
    cart_pole_env = gym.make('CartPole-v1')  # 環境作成
    cart_pole_observer = CartPoleObserver(cart_pole_env) # Observer作成
    ac_agent = ACAgent(actions = [0, 1]) # Agent作成
    cart_pole_trainer = CartPoleTrainer(num_episodes = 500, gamma = 0.9, buffer_length = 512) # Trainer作成
    cart_pole_trainer.train(agent = ac_agent, observer = cart_pole_observer, batch_size = 128) # 学習を実行
    cart_pole_trainer.save(ac_agent) # 学習済みのモデルを保存

    # 学習曲線の描画
    import matplotlib.pyplot as plt # matplotlib.pyplotのインポート

    plt.plot(cart_pole_trainer.rewards) # 報酬の折れ線グラフの描画
    plt.title('Train Curve', fontsize=20) # タイトルを設定
    plt.ylabel('Rewards', fontsize=20) # 縦軸のラベルを設定
    plt.xlabel('Episode', fontsize=20) # 横軸のラベルを指定
    plt.show() # グラフを表示
